<a href="https://colab.research.google.com/github/jdmartinev/ArtificialIntelligenceIM/blob/main/Lecture02/notebooks/L02_hyperparameter_tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import Ridge
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [2]:
# 1. Cargar la base de datos de Boston Housing
boston = fetch_openml(name="boston", version=1, as_frame=False, parser='liac-arff')
X, y = boston.data, boston.target

# 2. Separar un conjunto de prueba final (20%) que no tocaremos
# El resto (80%) se usará para entrenamiento y cross-validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Escalar las características (muy importante para RBFs)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f"Datos para entrenar y validar: {X_train.shape}")
print(f"Datos para prueba final: {X_test.shape}")

# Definir la función del kernel RBF que usaremos
def rbf_kernel(X1, X2, gamma_param):
    # np.newaxis ayuda a que las operaciones de broadcasting funcionen correctamente
    dist_sq = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
    return np.exp(-dist_sq / lambda_param)

Datos para entrenar y validar: (404, 13)
Datos para prueba final: (102, 13)


In [8]:
# --- Versión 1: K-Fold Manual ---

print("\n--- Iniciando K-Fold Manual ---")
gamma_candidates = [10, 50, 100]
n_centers = 50
ridge_alpha = 0.1
kf = KFold(n_splits=5, shuffle=True, random_state=42)
mean_errors = {}

for gamma_val in gamma_candidates:
    fold_errors = []
    for train_indices, val_indices in kf.split(X_train):
        X_train_fold, X_val_fold = X_train[train_indices], X_train[val_indices]
        y_train_fold, y_val_fold = y_train[train_indices], y_train[val_indices]
        kmeans = KMeans(n_clusters=n_centers, random_state=42, n_init='auto').fit(X_train_fold)
        centers = kmeans.cluster_centers_
        phi_train = rbf_kernel(X_train_fold, centers, gamma_val)
        phi_val = rbf_kernel(X_val_fold, centers, gamma_val)
        model = Ridge(alpha=ridge_alpha).fit(phi_train, y_train_fold)
        error = mean_squared_error(y_val_fold, model.predict(phi_val))
        fold_errors.append(error)
    mean_errors[gamma_val] = np.mean(fold_errors)

best_gamma_manual = min(mean_errors, key=mean_errors.get)
print(f"\nMejor gamma encontrado manualmente: {best_gamma_manual}")

# --- NOVEDAD: Re-entrenamiento y Evaluación Final para el Modelo Manual ---
print("\n--- Evaluación Final del Mejor Modelo (Manual) ---")
# 1. Aprender los centros usando TODOS los datos de entrenamiento
kmeans_final = KMeans(n_clusters=n_centers, random_state=42, n_init='auto').fit(X_train)
final_centers = kmeans_final.cluster_centers_

# 2. Transformar el conjunto de entrenamiento completo y el de prueba
phi_train_final = rbf_kernel(X_train, final_centers, best_gamma_manual)
phi_test_final = rbf_kernel(X_test, final_centers, best_gamma_manual)

# 3. Entrenar el modelo final con TODOS los datos de entrenamiento
final_model_manual = Ridge(alpha=ridge_alpha)
final_model_manual.fit(phi_train_final, y_train)

# 4. Reportar el error en el conjunto de prueba
final_predictions = final_model_manual.predict(phi_test_final)
final_mse = mean_squared_error(y_test, final_predictions)
print(f"Error Cuadrático Medio en el conjunto de prueba final (Manual): {final_mse:.2f}")



--- Iniciando K-Fold Manual ---

Mejor gamma encontrado manualmente: 50

--- Evaluación Final del Mejor Modelo (Manual) ---
Error Cuadrático Medio en el conjunto de prueba final (Manual): 15.99


In [9]:
# --- Versión 2: GridSearchCV de Scikit-learn ---

print("\n--- Iniciando GridSearchCV de Scikit-learn ---")

# --- NOVEDAD: RBFTransformer modificado para usar los datos como centros ---
class RBFDataAsCentersTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, gamma_val=1.0):
        self.gamma_val = gamma_val

    def fit(self, X, y=None):
        # En lugar de K-Means, los centros son los propios datos de entrenamiento
        self.centers_ = X
        return self

    def transform(self, X, y=None):
        return rbf_kernel(X, self.centers_, self.gamma_val)

pipeline = Pipeline([
    ('rbf_transformer', RBFDataAsCentersTransformer()),
    ('ridge', Ridge())
])

# --- NOVEDAD: El grid ya no incluye n_centers ---
param_grid = {
    'rbf_transformer__gamma_val': [10, 50, 100, 200],
    'ridge__alpha': [0.1, 1.0, 10.0]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5,
                           scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)

print(f"\nMejores hiperparámetros (GridSearchCV): {grid_search.best_params_}")
print(f"Mejor score (MSE negativo) en CV: {grid_search.best_score_:.2f}")

# --- Evaluación Final del Mejor Modelo (GridSearchCV) ---
print("\n--- Evaluación Final en el Conjunto de Prueba (GridSearchCV) ---")
best_model_gs = grid_search.best_estimator_
final_predictions_gs = best_model_gs.predict(X_test)
final_mse_gs = mean_squared_error(y_test, final_predictions_gs)

print(f"Error Cuadrático Medio en el conjunto de prueba final (GridSearchCV): {final_mse_gs:.2f}")




--- Iniciando GridSearchCV de Scikit-learn ---
Fitting 5 folds for each of 12 candidates, totalling 60 fits

Mejores hiperparámetros (GridSearchCV): {'rbf_transformer__gamma_val': 10, 'ridge__alpha': 0.1}
Mejor score (MSE negativo) en CV: -12.65

--- Evaluación Final en el Conjunto de Prueba (GridSearchCV) ---
Error Cuadrático Medio en el conjunto de prueba final (GridSearchCV): 11.91
